In [8]:
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.genai as genai
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [9]:
get_ipython().system('pip install --upgrade google-generativeai httpx')
print("✅ `google-generativeai` and `httpx` have been upgraded to their latest versions.")

✅ `google-generativeai` and `httpx` have been upgraded to their latest versions.


In [12]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library by setting the API key as an environment variable
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [13]:
# A Helper Function to Run Our Agents

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [14]:
#  Agent Definitions for our Specialist Team
#  Agent Definition

day_trip_agent = Agent(
    name="day_trip_agent",
    model="gemini-2.5-flash",
    description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
    instruction="""
    You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

    Your Mission:
    Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

    Guidelines:
    1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
    2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
    3. **Real-Time Focus**: Search for current operating hours and special events.
    4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

    RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
    """,
    tools=[google_search]
)

foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an expert food critic. Your goal is to find the absolute best food, restaurants, or culinary experiences based on a user's request. When you recommend a place, state its name clearly. For example: 'The best sushi is at **Jin Sho**.'"
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a navigation assistant. Given a starting point and a destination, provide clear directions on how to get from the start to the end."
)

# --- The Brain of the Operation: The Router Agent ---
# We update the router's instructions to know about the new 'combo' task.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_combo': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We'll create a dictionary of all our individual worker agents
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "transportation_agent": transportation_agent, # Add the new agent!
}

print("🤖 Agent team assembled for sequential workflows!")

🤖 Agent team assembled for sequential workflows!


In [15]:
# --- Let's Test the Sequential Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the COMBO
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route == 'find_and_navigate_combo':
            print("\n--- Starting Find and Navigate Combo Workflow ---")

            # STEP 2a: Run the foodie_agent first
            foodie_session = await session_service.create_session(app_name=foodie_agent.name, user_id=my_user_id)
            foodie_response = await run_agent_query(foodie_agent, query, foodie_session, my_user_id)

            # STEP 2b: Extract the destination from the first agent's response
            # (This is a simple regex, a more robust solution might use a structured output format)
            match = re.search(r'\*\*(.*?)\*\*', foodie_response)
            if not match:
                print("🚨 Could not determine the restaurant name from the response.")
                continue
            destination = match.group(1)
            print(f"💡 Extracted Destination: {destination}")

            # STEP 2c: Create a new query and run the transportation_agent
            directions_query = f"Give me directions to {destination} from the Palo Alto Caltrain station."
            print(f"\n🗣️ New Query for Transport Agent: '{directions_query}'")
            transport_session = await session_service.create_session(app_name=transportation_agent.name, user_id=my_user_id)
            await run_agent_query(transportation_agent, directions_query, transport_session, my_user_id)

            print("--- Combo Workflow Complete ---")

        elif chosen_route in worker_agents:
            # This is a simple, single-agent route
            worker_agent = worker_agents[chosen_route]
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()


🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'bb96af96-7e66-49eb-ae75-0a010749e293'...
🚦 Router has selected route: 'foodie_agent'

🚀 Running query for agent: 'foodie_agent' in session: 'fec84c4a-6bcc-4985-9c7b-5a5b9ceff947'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""For the absolute best sushi experience in Palo Alto, I recommend **Iki Omakase**.

Iki Omakase is praised for its "Refined Modern Omakase" concept, led by Chef Jiabo Li, whose culinary philosophy is deeply rooted in his journey as a dedicated sushi shokunin. The restaurant focuses on culinary distinctiveness, elevating service, and expanding flavor profiles, using the finest globally and locally sourced ingredients to present culinary creations that celebrate sushi as an art form. Chef Li honed his skills at prestigious sushi establishments in Sa

For the absolute best sushi experience in Palo Alto, I recommend **Iki Omakase**.

Iki Omakase is praised for its "Refined Modern Omakase" concept, led by Chef Jiabo Li, whose culinary philosophy is deeply rooted in his journey as a dedicated sushi shokunin. The restaurant focuses on culinary distinctiveness, elevating service, and expanding flavor profiles, using the finest globally and locally sourced ingredients to present culinary creations that celebrate sushi as an art form. Chef Li honed his skills at prestigious sushi establishments in San Francisco before opening Iki Omakase, where he aims to deliver an unforgettable omakase experience with a unique Bay Area style and a firm foundation in Edomae sushi.

While Iki Omakase offers a premium experience with a prix fixe menu, other highly regarded sushi establishments in Palo Alto include:

*   **Jin Sho**: Known for exquisite Japanese cuisine from co-owners who were formerly chefs at Nobu in New York. It's noted for its tender care in preparing a unique fish selection, resulting in beautifully crafted sushi dishes.
*   **Kanpai Sushi**: An independently-owned Japanese restaurant serving authentic cuisine. It has a strong recent recommendation from a user who would bring work colleagues from Tokyo, indicating high quality and authenticity.
*   **Sushi Tomi**: A cozy spot recognized for its fresh fish and traditional sushi offerings, gaining popularity through word-of-mouth for its chef's choice meals and authentic atmosphere.

--------------------------------------------------


🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '7996bc51-72e4-44ec-8f26-c026b99960c5'...
🚦 Router has selected route: 'weekend_guide_agent'

🚀 Running query for agent: 'weekend_guide_agent' in session: 'b7a6ebc3-62ba-4975-83f7-f78e7c170a03'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Here are some cool outdoor concerts and festivals happening this weekend, December 27-28, 2025:

**In Florida, USA:**

*   **We Belong Here: Palm Beach 2025** is a two-day outdoor festival taking place at the Meyer Amphitheatre in West Palm Beach on Saturday, December 27th, and Sunday, December 28th.
*   The **Frontyard Holiday Festival** in Orlando offers 80 live outdoor performances. This free event at the Dr. Phillips Center runs throughout the weekend, from 11:00 AM to 1

Here are some cool outdoor concerts and festivals happening this weekend, December 27-28, 2025:

**In Florida, USA:**

*   **We Belong Here: Palm Beach 2025** is a two-day outdoor festival taking place at the Meyer Amphitheatre in West Palm Beach on Saturday, December 27th, and Sunday, December 28th.
*   The **Frontyard Holiday Festival** in Orlando offers 80 live outdoor performances. This free event at the Dr. Phillips Center runs throughout the weekend, from 11:00 AM to 11:00 PM on both Saturday, December 27th, and Sunday, December 28th.

**In Amsterdam, Netherlands:**

*   **The Night Sky** is a two-night outdoor closing celebration happening at Westweelde on Saturday, December 27th, and Sunday, December 28th. This event focuses on dance, connection, and reflection.

While some venues like Garden AMP in Garden Grove, CA, and Soaring Eagle Casino & Resort in Michigan are known for outdoor concerts, the specific listings for this weekend at those locations do not explicitly state that the performances on December 27th and 28th will be outdoors. Similarly, ArtsQuest in Bethlehem, PA, hosts large outdoor festivals like Musikfest, but its listed concerts for this weekend are in the Musikfest Café.

--------------------------------------------------


🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '54a62ba3-9b56-49ec-860f-f8b564279d71'...
🚦 Router has selected route: 'find_and_navigate_combo'

--- Starting Find and Navigate Combo Workflow ---

🚀 Running query for agent: 'foodie_agent' in session: 'ffa72346-2581-410c-bad3-654410e01ab1'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""The absolute best sushi in Palo Alto can be found at **Jin Sho**. This restaurant is consistently praised for its high-quality ingredients and exceptional presentation, with some even calling it a "Steve Jobs' favorite sushi." While it can be on the pricier side, the culinary experience is considered amazing. Jin Sho offers a variety of Japanese dishes, including sushi omakase, and

The absolute best sushi in Palo Alto can be found at **Jin Sho**. This restaurant is consistently praised for its high-quality ingredients and exceptional presentation, with some even calling it a "Steve Jobs' favorite sushi." While it can be on the pricier side, the culinary experience is considered amazing. Jin Sho offers a variety of Japanese dishes, including sushi omakase, and is known for its yellowtail jalapeno appetizer and black cod entree.

**To get to Jin Sho from the Caltrain Station, you'll want to head to the California Avenue Caltrain Station.**

Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306. The California Avenue Caltrain Station is incredibly close to the restaurant, with sources indicating it's just about 91 yards away, a mere 2-minute walk.

Once you arrive at the California Avenue Caltrain Station, Jin Sho will be a very short walk away on California Avenue.

--------------------------------------------------

💡 Extracted Destination: Jin Sho

🗣️ New Query for Transport Agent: 'Give me directions to Jin Sho from the Palo Alto Caltrain station.'

🚀 Running query for agent: 'transportation_agent' in session: '71308aa1-43e0-4c08-b562-e529da65c090'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""To get to Jin Sho from the Palo Alto Caltrain stations, the closest option is the California Avenue Caltrain Station.

**From California Avenue Caltrain Station to Jin Sho (Walking)**

Jin Sho is conveniently located on California Avenue, a short walk from the California Avenue Caltrain Station.

1.  Exit the California Avenue Caltrain Station at 101 California Avenue, Palo Alto, CA 94306.
2.  Walk southeast on California Avenue towards 454 California Avenue. Jin Sho will be on your right.

The walk is approximately 91-108 yards and should take about 2-3 minutes.

**From Palo Alto Caltrain Station (Universit

To get to Jin Sho from the Palo Alto Caltrain stations, the closest option is the California Avenue Caltrain Station.

**From California Avenue Caltrain Station to Jin Sho (Walking)**

Jin Sho is conveniently located on California Avenue, a short walk from the California Avenue Caltrain Station.

1.  Exit the California Avenue Caltrain Station at 101 California Avenue, Palo Alto, CA 94306.
2.  Walk southeast on California Avenue towards 454 California Avenue. Jin Sho will be on your right.

The walk is approximately 91-108 yards and should take about 2-3 minutes.

**From Palo Alto Caltrain Station (University Avenue) to Jin Sho (Driving or Public Transport)**

If you arrive at the main Palo Alto Caltrain Station at 95 University Avenue, Palo Alto, CA 94301, you will need to travel a bit further to reach Jin Sho.

*   **Driving:** The driving distance from the Palo Alto Caltrain Station (University Avenue) to Jin Sho is approximately 2 miles. It will take around 5-10 minutes depending on traffic.
*   **Public Transport:** You can take a local bus service from the Palo Alto Caltrain Station towards California Avenue. Several VTA bus lines (such as the 22 and 522) serve the area and connect to the California Avenue transit stops.

--------------------------------------------------

--- Combo Workflow Complete ---


In [16]:
# --- Agent Definitions for our Specialist Team (Refactored for Sequential Workflow) ---

# ✨ CHANGE 1: We tell foodie_agent to save its output to the shared state.
# Note the new `output_key` and the more specific instruction.
foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment and nothing else.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="destination"  # ADK will save the agent's final response to state['destination']
)

# ✨ CHANGE 2: We tell transportation_agent to read from the shared state.
# The `{destination}` placeholder is automatically filled by the ADK from the state.
transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are a navigation assistant. Given a destination, provide clear directions.
    The user wants to go to: {destination}.

    Analyze the user's full original query to find their starting point.
    Then, provide clear directions from that starting point to {destination}.
    """,
)

# ✨ CHANGE 3: Define the SequentialAgent to manage the workflow.
# This agent will run foodie_agent, then transportation_agent, in that exact order.
find_and_navigate_agent = SequentialAgent(
    name="find_and_navigate_agent",
    sub_agents=[foodie_agent, transportation_agent],
    description="A workflow that first finds a location and then provides directions to it."
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

# --- The Brain of the Operation: The Router Agent ---

# We update the router to know about our new, powerful SequentialAgent.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_agent': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We create a dictionary of all our executable agents for easy lookup.
# This now includes our powerful new workflow agent!
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "find_and_navigate_agent": find_and_navigate_agent, # Add the new sequential agent
}

print("🤖 Agent team assembled with a SequentialAgent workflow!")

🤖 Agent team assembled with a SequentialAgent workflow!


In [17]:
# --- Let's Test the Streamlined Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the SequentialAgent
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        # This logic is now much simpler! The SequentialAgent is treated just like any other worker.
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()


🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'be2c2105-2799-4593-a6ee-9e9cadbd3c63'...
🚦 Router has selected route: 'foodie_agent'
--- Handing off to foodie_agent ---

🚀 Running query for agent: 'foodie_agent' in session: '74c1352f-3a3a-40f3-9695-e6f6fd0b9d3c'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text='Jin Sho'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webki

Jin Sho

--------------------------------------------------

--- foodie_agent Complete ---

🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '9fdaa9b8-9abe-4d98-8fc4-570f6edb201b'...
🚦 Router has selected route: 'weekend_guide_agent'
--- Handing off to weekend_guide_agent ---

🚀 Running query for agent: 'weekend_guide_agent' in session: 'c649ca7c-ab5c-4904-b051-79953cd26904'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""This weekend, December 27-29, 2025, there are several music events and festivals happening around the world, particularly in regions experiencing summer, making outdoor concerts a possibility.

Here are a few notable events that may feature outdoor music:

**In South Africa:**

*   **Nkomazi Farmers Market - Chills in Summer** will take place at the Mahushe Shongwe Nature Reserve Main Camp in Mpumalanga

This weekend, December 27-29, 2025, there are several music events and festivals happening around the world, particularly in regions experiencing summer, making outdoor concerts a possibility.

Here are a few notable events that may feature outdoor music:

**In South Africa:**

*   **Nkomazi Farmers Market - Chills in Summer** will take place at the Mahushe Shongwe Nature Reserve Main Camp in Mpumalanga on Saturday, December 27, 2025. This event is described as a food, picnic, and family gathering, which often includes outdoor music or entertainment.
*   The **MAKARAPA DAY 7TH ANNUAL** is scheduled for Sunday, December 28, 2025, in Ga-Nchabeleng, Limpopo. This cultural and music festival is highly likely to have outdoor performances.
*   **MATJHABENG SOULFUL BLOW** is happening at the Rob Cricket Club in Welkom, Free State, on Saturday, December 27, 2025, featuring soul music and dance, potentially in an outdoor setting.
*   **Master Kg Homecoming** will be held at the Maruleng Showground in Limpopo on Saturday, December 27, 2025, focusing on music, Amapiano, and dance, with a showground location suggesting an outdoor event.

**In the United States:**

*   **We Belong Here Palm Beach 2025** is a music festival in West Palm Beach, Florida, running from December 27-28, 2025. Florida's warm December climate makes outdoor elements highly probable for this event.
*   In Fort Worth, Texas, **Christmas in the Garden at the Fort Worth Botanic Garden** runs from December 27, 2025, to January 5, 2026. While primarily a light display, such events often feature live outdoor music.
*   The **Jackpot NYE 2025** music festival is in Las Vegas, Nevada, from December 26-27, 2025. Many New Year's Eve lead-up events in Las Vegas incorporate outdoor stages.
*   **HiJinx Festival 2025** in Philadelphia, Pennsylvania, on December 27-28, 2025, is a music festival that may have outdoor components despite the winter season.

**In Australia and New Zealand:**

*   **Beyond The Valley 2025** is a music festival in Lardner, Australia, from December 27, 2025, to January 1, 2026. Given it's summer in Australia, this is very likely an outdoor music festival.
*   The **Lakes Festival 2025** in Christchurch, New Zealand, on December 28, 2025, is likely to be an outdoor event due to the summer season there.
*   **Rock the Bowl Festival 2025** in New Plymouth, New Zealand, on December 29, 2025, implies an outdoor amphitheater setting.

If you have a specific location in mind, I can try to find more tailored results for outdoor concerts near you!

--------------------------------------------------

--- weekend_guide_agent Complete ---

🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '1ab955ff-a036-444b-ad3d-e265925844bf'...
🚦 Router has selected route: 'find_and_navigate_agent'
--- Handing off to find_and_navigate_agent ---

🚀 Running query for agent: 'find_and_navigate_agent' in session: 'eec9818c-8c97-4c6d-b06d-e5c3a0b590bd'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Jin Sho

To get to Jin Sho from the Caltrain station, go to the California Avenue Caltrain Station. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306.

The California Avenue Caltrain station is just a short walk from Jin Sho. After exiting the station, head to California Avenue; Jin Sho is located directly on this street.""

To get to Jin Sho from the Caltrain station, head to the California Avenue Caltrain Station. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306.

After exiting the California Avenue Caltrain Station, Jin Sho is just a short walk away, located directly on California Avenue.

--------------------------------------------------

--- find_and_navigate_agent Complete ---


In [18]:
# --- Agent Definitions for an Iterative Workflow ---

# A tool to signal that the loop should terminate
COMPLETION_PHRASE = "The plan is feasible and meets all constraints."
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the plan is approved, signaling the loop should end."""
  print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  return {}

# Agent 1: Proposes an initial plan
planner_agent = Agent(
    name="planner_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a trip planner. Based on the user's request, propose a single activity and a single restaurant. Output only the names, like: 'Activity: Exploratorium, Restaurant: La Mar'.",
    output_key="current_plan"
)

# Agent 2 (in loop): Critiques the plan
critic_agent = Agent(
    name="critic_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction=f"""You are a logistics expert. Your job is to critique a travel plan. The user has a strict constraint: total travel time must be short.
    Current Plan: {{current_plan}}
    Use your tools to check the travel time between the two locations.
    IF the travel time is over 45 minutes, provide a critique, like: 'This plan is inefficient. Find a restaurant closer to the activity.'
    ELSE, respond with the exact phrase: '{COMPLETION_PHRASE}'""",
    output_key="criticism"
)

# Agent 3 (in loop): Refines the plan or exits
refiner_agent = Agent(
    name="refiner_agent", model="gemini-2.5-flash", tools=[google_search, exit_loop],
    instruction=f"""You are a trip planner, refining a plan based on criticism.
    Original Request: {{session.query}}
    Critique: {{criticism}}
    IF the critique is '{COMPLETION_PHRASE}', you MUST call the 'exit_loop' tool.
    ELSE, generate a NEW plan that addresses the critique. Output only the new plan names, like: 'Activity: de Young Museum, Restaurant: Nopa'.""",
    output_key="current_plan"
)

# ✨ The LoopAgent orchestrates the critique-refine cycle ✨
refinement_loop = LoopAgent(
    name="refinement_loop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3
)

# ✨ The SequentialAgent puts it all together ✨
iterative_planner_agent = SequentialAgent(
    name="iterative_planner_agent",
    sub_agents=[planner_agent, refinement_loop],
    description="A workflow that iteratively plans and refines a trip to meet constraints."
)

print("🤖 Agent team updated with an iterative LoopAgent workflow!")

🤖 Agent team updated with an iterative LoopAgent workflow!


In [19]:
# --- Agent Definitions for a Parallel Workflow ---

# Specialist Agent 1
museum_finder_agent = Agent(
    name="museum_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a museum expert. Find the best museum based on the user's query. Output only the museum's name.",
    output_key="museum_result"
)

# Specialist Agent 2
concert_finder_agent = Agent(
    name="concert_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are an events guide. Find a concert based on the user's query. Output only the concert name and artist.",
    output_key="concert_result"
)

# We can reuse our foodie_agent for the third parallel task!
# Just need to give it a new output_key for this workflow.
# restaurant_finder_agent = foodie_agent.copy(update={"output_key": "restaurant_result"})
restaurant_finder_agent = Agent(
    name="restaurant_finder_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="restaurant_result" # Set the correct output key for this workflow
)


# ✨ The ParallelAgent runs all three specialists at once ✨
parallel_research_agent = ParallelAgent(
    name="parallel_research_agent",
    sub_agents=[museum_finder_agent, concert_finder_agent, restaurant_finder_agent]
)

# Agent to synthesize the parallel results
synthesis_agent = Agent(
    name="synthesis_agent", model="gemini-2.5-flash",
    instruction="""You are a helpful assistant. Combine the following research results into a clear, bulleted list for the user.
    - Museum: {museum_result}
    - Concert: {concert_result}
    - Restaurant: {restaurant_result}
    """
)

# ✨ The SequentialAgent runs the parallel search, then the synthesis ✨
parallel_planner_agent = SequentialAgent(
    name="parallel_planner_agent",
    sub_agents=[parallel_research_agent, synthesis_agent],
    description="A workflow that finds multiple things in parallel and then summarizes the results."
)

print("🤖 Agent team supercharged with a ParallelAgent workflow!")

🤖 Agent team supercharged with a ParallelAgent workflow!


In [20]:
# --- The ULTIMATE Router Agent --- #

router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a master request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about finding a single food place.
    - 'find_and_navigate_agent': For queries that ask to *first find a place* and *then get directions* to it.
    - 'iterative_planner_agent': For planning a trip with a specific constraint that needs checking, like travel time.
    - 'parallel_planner_agent': For queries that ask to find multiple, independent things at once (e.g., a museum AND a concert AND a restaurant).
    - 'day_trip_agent': A general planner for any other simple day trip requests.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# The master dictionary of all our executable agents and workflows
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent, # For simple food queries
    "find_and_navigate_agent": find_and_navigate_agent, # Sequential
    "iterative_planner_agent": iterative_planner_agent, # Loop
    "parallel_planner_agent": parallel_planner_agent,   # Parallel
}

# --- Let's Test Everything! ---

async def run_fully_loaded_app():
    queries = [
        # Test Case 1: Simple Sequential Flow
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.",

        # Test Case 2: Iterative Loop Flow
        "Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.",

        # Test Case 3: Parallel Flow
        "Help me plan a trip to SF. I need one museum, one concert, and one great restaurant."
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_fully_loaded_app()


🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '6dfe927f-652b-4948-8a39-0d85530a9270'...
🚦 Router has selected route: 'find_and_navigate_agent'
--- Handing off to find_and_navigate_agent ---

🚀 Running query for agent: 'find_and_navigate_agent' in session: '1344849d-690a-4d6f-83c9-4cdabafce98c'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Jin Sho

To get to Jin Sho from the California Avenue Caltrain station, you can simply walk. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306. The California Avenue Caltrain station is also on California Avenue, making it a very short distance. You would typically exit the station and head towards 454 California Avenue."""
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  grounding_chunks=[


Jin Sho is a highly-regarded option for sushi in Palo Alto.

To get to Jin Sho from the California Avenue Caltrain station, you can easily walk. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306. Since the California Avenue Caltrain station is also on California Avenue, the distance is very short. You should exit the station and head towards 454 California Avenue.

--------------------------------------------------

--- find_and_navigate_agent Complete ---

🗣️ Processing New Query: 'Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '290551b0-93bc-493a-aa31-3f31a82630bd'...
🚦 Router has selected route: 'iterative_planner_agent'
--- Handing off to iterative_planner_agent ---

🚀 Running query for agent: 'iterative_planner_agent' in session: 'af979891-8cf1-4eae-9a12-84c950e0e57c'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text='Activity: Exploratorium, Restaurant: La Mar'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-


--------------------------------------------------
✅ Final Response:


An error occurred: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Tool use with function calling is unsupported', 'status': 'INVALID_ARGUMENT'}}

--------------------------------------------------

--- iterative_planner_agent Complete ---

🗣️ Processing New Query: 'Help me plan a trip to SF. I need one museum, one concert, and one great restaurant.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '9948c01c-90ac-4760-9fa1-cfcdae73da9a'...
🚦 Router has selected route: 'parallel_planner_agent'
--- Handing off to parallel_planner_agent ---

🚀 Running query for agent: 'parallel_planner_agent' in session: '3d769168-18f2-476b-b0ee-0d89f1cd47d0'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text='California Academy of Sciences'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;

Here's a plan for your trip to San Francisco:

*   **Museum:** California Academy of Sciences
*   **Concert:** Thievery Corporation (18+ Event)
*   **Restaurant:** State Bird Provisions

--------------------------------------------------

--- parallel_planner_agent Complete ---
